### Objetivo: Desenvolver e treinar o Modelo 1 (Species Classifier)

Conteúdo:
- Preparação específica dos dados de espécies
- Arquitetura do modelo (Transfer Learning)
- Treinamento e validação
- Análise de performance (confusion matrix, métricas)
- Salvamento do modelo treinado
- Análise de erros e interpretabilidade

In [ ]:
#Carregar dados para modelo classificador de espécies

from utils import *

# Carregar configurações
config = carregar_configuracoes()
print(f"✅ Configurações carregadas:")
print(f"  📏 Image size: {config['img_height']}x{config['img_width']}")
print(f"  📦 Batch size: {config['batch_size']}")

dataset_especies = carregar_dataset_especies()
train_gen, val_gen, test_gen = criar_geradores(dataset_especies, config)

print(f"🌿 Dataset de espécies carregado:")
print(f"  Train: {len(dataset_especies['train']['X'])} imagens")
print(f"  Val: {len(dataset_especies['val']['X'])} imagens")

✅ Configurações carregadas:
  📏 Image size: 224x224
  📦 Batch size: 32
Found 14446 validated image filenames belonging to 3 classes.
Found 3096 validated image filenames belonging to 3 classes.
Found 3096 validated image filenames belonging to 3 classes.
🌿 Dataset de espécies carregado:
  Train: 14446 imagens
  Val: 3096 imagens
